# Web socket creation
https://www.codearmo.com/python-tutorial/bybit-websockets-python

In [1]:
%reload_ext autoreload
%autoreload 2

from pybit.unified_trading import HTTP
from pybit.unified_trading import WebSocket
import pybit # working source
import pandas as pd
import datetime 

import sys
import os

sys.path.append('../../src/') 
sys.path.append('../../../src/')

import src_API

from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from pybit.unified_trading import WebSocket
        
key = os.getenv('BYBIT_KEY_test')
secret = os.getenv('BYBIT_SECRET_test')

public = WebSocket(channel_type='linear', testnet=False)
private = WebSocket(channel_type='private',
                     api_key=key,
                     api_secret=secret, 
                     testnet=False) 

WebSocket Unified V5 (wss://stream.bybit.com/v5/public/linear) encountered error: Connection to remote host was lost..


In [4]:
orderbook = [] 
   
def handle_orderbook_message(message):
    '''
    custom on callback function for orderbook stream , insert given values
    in to dictionary to be appended to oderbook list. 
    None.

    '''
    
    data = message.get('data', None)
    
    global orderbook 
    
    current = {}
    if data:
        try:
            current['asset'] = data.get('s')
            current['best_bid'] = float(data.get('b')[0][0])
            current['bid_volume'] = float(data.get('b')[0][1])
            current['best_ask'] = float(data.get('a')[0][0])
            current['ask_volume'] = float(data.get('a')[0][1])
            orderbook.append(current)
        except (TypeError, IndexError, AttributeError):
            pass
        
        
public.orderbook_stream(depth=1, symbol='BTCUSDT', callback=handle_orderbook_message)    

import time
for _ in range(10):
    print(len(orderbook))
    time.sleep(1)
    print(orderbook[-1])

0
{'asset': 'BTCUSDT', 'best_bid': 79116.2, 'bid_volume': 8.265, 'best_ask': 79116.3, 'ask_volume': 0.185}
39
{'asset': 'BTCUSDT', 'best_bid': 79116.2, 'bid_volume': 5.021, 'best_ask': 79116.3, 'ask_volume': 2.822}
81
{'asset': 'BTCUSDT', 'best_bid': 79122.6, 'bid_volume': 7.155, 'best_ask': 79122.7, 'ask_volume': 1.401}
137
{'asset': 'BTCUSDT', 'best_bid': 79125.1, 'bid_volume': 4.908, 'best_ask': 79125.2, 'ask_volume': 1.375}
189
{'asset': 'BTCUSDT', 'best_bid': 79125.1, 'bid_volume': 3.308, 'best_ask': 79125.2, 'ask_volume': 5.361}
225
{'asset': 'BTCUSDT', 'best_bid': 79125.1, 'bid_volume': 0.116, 'best_ask': 79125.2, 'ask_volume': 7.224}
276
{'asset': 'BTCUSDT', 'best_bid': 79125.1, 'bid_volume': 0.435, 'best_ask': 79125.2, 'ask_volume': 12.204}
330
{'asset': 'BTCUSDT', 'best_bid': 79122.5, 'bid_volume': 0.175, 'best_ask': 79122.6, 'ask_volume': 8.722}
380
{'asset': 'BTCUSDT', 'best_bid': 79122.5, 'bid_volume': 0.08, 'best_ask': 79122.6, 'ask_volume': 12.362}
414
{'asset': 'BTCUSDT

In [4]:
import json
from kafka import KafkaProducer
import time

# Настройка Kafka продюсера
producer = KafkaProducer(
    bootstrap_servers='localhost:29092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

orderbook = []

def handle_orderbook_message(message):
    '''
    Callback для обработки сообщений ордербука.
    Данные сохраняются в orderbook и отправляются в Kafka.
    '''
    
    data = message.get('data', None)
    global orderbook
    
    current = {}
    if data:
        try:
            # Получение данных из сообщения
            current['asset'] = data.get('s')
            current['best_bid'] = float(data.get('b')[0][0])
            current['bid_volume'] = float(data.get('b')[0][1])
            current['best_ask'] = float(data.get('a')[0][0])
            current['ask_volume'] = float(data.get('a')[0][1])
            
            # Добавление текущего состояния в orderbook
            orderbook.append(current)
            
            # Отправка текущего состояния в Kafka
            producer.send('orderbook_data', value=current)
        
        except (TypeError, IndexError, AttributeError):
            pass

# Запуск стрима ордербука с передачей callback функции
public.orderbook_stream(depth=1, symbol='BTCUSDT', callback=handle_orderbook_message)

# Периодическая проверка содержимого orderbook (для демонстрации)
for _ in range(10):
    print(len(orderbook))
    time.sleep(1)
    if orderbook:
        print(orderbook[-1])

0
{'asset': 'BTCUSDT', 'best_bid': 76189.9, 'bid_volume': 9.212, 'best_ask': 76190.0, 'ask_volume': 1.894}
31
{'asset': 'BTCUSDT', 'best_bid': 76189.9, 'bid_volume': 5.14, 'best_ask': 76190.0, 'ask_volume': 4.354}
58
{'asset': 'BTCUSDT', 'best_bid': 76189.9, 'bid_volume': 0.769, 'best_ask': 76190.0, 'ask_volume': 5.383}
70
{'asset': 'BTCUSDT', 'best_bid': 76187.3, 'bid_volume': 0.046, 'best_ask': 76187.4, 'ask_volume': 4.691}
101
{'asset': 'BTCUSDT', 'best_bid': 76166.8, 'bid_volume': 4.289, 'best_ask': 76166.9, 'ask_volume': 0.315}
165
{'asset': 'BTCUSDT', 'best_bid': 76166.8, 'bid_volume': 5.425, 'best_ask': 76166.9, 'ask_volume': 0.169}
180
{'asset': 'BTCUSDT', 'best_bid': 76175.7, 'bid_volume': 13.219, 'best_ask': 76175.8, 'ask_volume': 1.501}
216
{'asset': 'BTCUSDT', 'best_bid': 76175.7, 'bid_volume': 9.726, 'best_ask': 76175.8, 'ask_volume': 1.508}
258
{'asset': 'BTCUSDT', 'best_bid': 76175.7, 'bid_volume': 9.851, 'best_ask': 76175.8, 'ask_volume': 1.824}
302
{'asset': 'BTCUSDT',

In [ ]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    'orderbook_data',
    bootstrap_servers='localhost:29092',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

for message in consumer:
    print(message.value)


{'asset': 'BTCUSDT', 'best_bid': 76136.0, 'bid_volume': 4.29, 'best_ask': 76136.1, 'ask_volume': 13.091}
{'asset': 'BTCUSDT', 'best_bid': 76136.0, 'bid_volume': 4.29, 'best_ask': 76136.1, 'ask_volume': 12.838}
{'asset': 'BTCUSDT', 'best_bid': 76136.0, 'bid_volume': 4.266, 'best_ask': 76136.1, 'ask_volume': 12.838}
{'asset': 'BTCUSDT', 'best_bid': 76136.0, 'bid_volume': 4.216, 'best_ask': 76136.1, 'ask_volume': 12.838}
{'asset': 'BTCUSDT', 'best_bid': 76136.0, 'bid_volume': 4.216, 'best_ask': 76136.1, 'ask_volume': 12.714}
{'asset': 'BTCUSDT', 'best_bid': 76136.0, 'bid_volume': 4.3, 'best_ask': 76136.1, 'ask_volume': 12.714}
{'asset': 'BTCUSDT', 'best_bid': 76136.0, 'bid_volume': 4.3, 'best_ask': 76136.1, 'ask_volume': 12.838}
{'asset': 'BTCUSDT', 'best_bid': 76136.0, 'bid_volume': 4.28, 'best_ask': 76136.1, 'ask_volume': 12.838}
{'asset': 'BTCUSDT', 'best_bid': 76136.0, 'bid_volume': 4.28, 'best_ask': 76136.1, 'ask_volume': 12.837}
{'asset': 'BTCUSDT', 'best_bid': 76136.0, 'bid_volume'

KeyboardInterrupt: 

WebSocket Unified V5 (wss://stream.bybit.com/v5/public/linear) encountered error: Connection to remote host was lost..
